In [ ]:
from collections import namedtuple
import pickle
import re
import numpy as np
import pandas as pd
import tweepy
import networkx as nx

In [ ]:
ExtendedUser = namedtuple('extended_user', ['user', 'tweets'])

### Retweet network

In [ ]:
actor_types = pd.read_csv('Categorized actors.csv')

def collapse_categories(category):
    if category in ['meps', 'europarties']:
        return 'meps & europarties'
    elif category in ['politicians', 'prime_ministers', 'climate_ministers', 'eu_commissioners']:
        return 'assorted politicians'
    elif category in ['eu departments', 'governmental institutions']:
        return 'public institutions'
    else:
        return category

actor_types['collapsed_category'] = actor_types.category.apply(collapse_categories)
actor_types = actor_types.set_index('usernames').to_dict()['collapsed_category']

In [ ]:
def include(retweeted, retweeter, included_actors):
    if retweeted != retweeter:
        if retweeter in included_actors and retweeted in included_actors:
            return True
        else:
            return False
    else:
        return False

In [ ]:
# Retweeted
from collections import Counter
with open("../Data/df_tweets_selected_users_after_oct.p", 'rb') as f:
    df_retweets = pickle.load(f)
df_retweets = df_retweets[~pd.isna(df_retweets['retweeted'])]
df_retweets = df_retweets[
    df_retweets.apply(lambda x: include(x['retweeted'], x['user.screen_name'], actor_types), axis=1)
]
edge_list = [tuple(tup) for tup in df_retweets[['user.screen_name', 'retweeted']].itertuples(index=False)]
edge_dict = [(a, b, {'weight': w}) for (a, b), w in Counter(edge_list).items()]
G = nx.DiGraph(edge_dict)
node_category = {actor:cat for actor, cat in actor_types.items()
                 if actor in list( G.nodes )}
nx.set_node_attributes(G, node_category, 'category')
nx.write_gexf(G, "retweeted.gexf")

In [ ]:
# Should all be equal
print( len({*set(e[0] for e in edge_list), *set(e[1] for e in edge_list)}) )
print( len(G.nodes) )
print( len(node_category) )

540
540
540
